In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('SBIN.NS.txt')

In [2]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
61,2020-09-24,181.100006,181.800003,175.500000,176.350006,176.350006,46114798
62,2020-09-25,179.000000,182.750000,176.149994,182.199997,182.199997,46550127
63,2020-09-28,184.000000,188.149994,183.350006,187.250000,187.250000,41921548
64,2020-09-29,188.500000,188.949997,183.250000,185.050003,185.050003,35688726
65,2020-09-30,185.050003,186.800003,182.399994,185.399994,185.399994,37641083


In [3]:
####################################
##########   Question 1  ###########
####################################


def predict(prices, times, num_sims):
    
    if times[0] != 0:
        times = [0] + list(times)
    
    prices = np.array(prices)
    us = np.log(prices[1:]/prices[:-1]) 
    n = len(prices) - 1
    Eu = sum(us)/float(n)
    sigma2 = np.sum((us - Eu)**2)/(n-1)
    sigma = np.sqrt(sigma2)
    mu = Eu + sigma2/2.0
    
#     print(f'mu = {mu:.3e}\t sigma^2 = {sigma2:.3e}')
    
    predictions = []
    for _ in range(num_sims):
        predictions.append([])
        lS = prices[-1]
        for i in range(1, len(times)):
            dt = times[i] - times[i-1]
            S = lS * np.exp((mu - sigma2/2)*dt + sigma*np.random.normal()*np.sqrt(dt))
            predictions[-1].append(S)
            lS = S
    
    return predictions

    
prices = data['Close'].to_numpy()
T, N, M, K = 30, 300, 1000, 1.1*prices[-1]

asian_prices = []
for pred in predict(prices, np.linspace(0, T, N, endpoint=True), M):
    asian_prices.append( max(K - sum(pred)/len(pred), 0.0) )

asian_prices = np.array(asian_prices)
muh = asian_prices.mean()
sigh = asian_prices.std()
print(f'mu^ = {muh}')
print(f'sigma^ = {sigh}')
print(f'95% confidence interval = ({muh - 1.96*sigh/np.sqrt(M): .3f}, {muh + 1.96*sigh/np.sqrt(M): .3f})')

mu^ = 18.267498051173394
sigma^ = 12.064316544851978
95% confidence interval = ( 17.520,  19.015)


In [5]:
####################################
##########   Question 2  ###########
####################################


Y = []
X = []
# expectation of european option
temp_sim = K - np.array(predict(prices, [0, T], M)).ravel()
temp_sim[temp_sim < 0] = 0
EX = temp_sim.mean()
for pred in predict(prices, np.linspace(0, T, N), M):
    y = max(K - sum(pred)/len(pred), 0.0)
    x = max(K - pred[-1], 0)
    Y.append(y)
    X.append(x)

X = np.array(X)
Y = np.array(Y)
b = np.sum((X-X.mean())*(Y-Y.mean()))/np.sum((X-X.mean())**2) 

Yb = np.array([Y[i] - b*(X[i] - EX) for i in range(len(X))])


cv_muh = Yb.mean()
cv_sigh = Yb.std()
# print(f'mu^ = {Yb.mean()}  ; sigma^ = {Yb.std()}')
print(f'mu^ = {cv_muh}')
print(f'sigma^ = {cv_sigh}')
print(f'95% confidence interval = ({cv_muh - 1.96*cv_sigh/np.sqrt(M): .3f}, {cv_muh + 1.96*cv_sigh/np.sqrt(M): .3f})')

mu^ = 18.74958432647338
sigma^ = 6.369083329171708
95% confidence interval = ( 18.355,  19.144)
